In [1]:
# -- WeatherPy Challenge, Vacation Search
# -- Challenge Part 2: Have Customers Narrow 
#    Their Travel Searches Based on Temperature 
#    and Precipitation

# -- Refactored and consolidated code throughout for a cleaner file. 
# -- Total Cells reduced to 7.

# -- Importing and loading dependencies.

# Import the dependencies.
import pandas as pd
import gmaps
import requests 
import os 

# Import the API key.
from config import g_key 

# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

# Store the CSV you saved created in Part 1 into a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_Challenge.csv")

# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [2]:
# -- Prompts for customer input for Min Temp, Max Temp, 
#    and if desired weather of raining or snowing.

# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))

max_temp = float(input("What is the maximum temperature you would like for your trip? ")) 

# -- Ask the customer if they would like it to be raining or not.
rain_selection = input("Do you want it to be raining? (yes/no) ")

# -- Ask the customer if they would like it to be snowing or not.
snow_selection = input("Do you want it to be snowing? (yes/no) ")

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [3]:
# -- Statements to filter data based on customer conditions:
#    Resource: https://stackoverflow.com/questions/18194404/create-column-with-elif-in-pandas 

if (rain_selection == "yes"):
    rain_preference = (city_data_df["Rain Inches (Last 3 hrs)"] > 0)
    
elif (rain_selection == "no"):
    rain_preference = (city_data_df["Rain Inches (Last 3 hrs)"] == 0)
    
if (snow_selection == "yes"):
    snow_preference = (city_data_df["Snow Inches (Last 3 hrs)"] > 0)
    
elif (snow_selection == "no"):
    snow_preference = (city_data_df["Snow Inches (Last 3 hrs)"] == 0)

# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp) & 
                                       rain_preference &
                                       snow_preference
                                      ]

# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""

hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Dingle,PH,82.08,scattered clouds,11.00,122.67,
9,Chadiza,ZM,84.70,few clouds,-14.07,32.44,
11,Avarua,CK,71.60,overcast clouds,-21.21,-159.78,
17,Inhambane,MZ,87.80,broken clouds,-23.86,35.38,
21,Georgetown,MY,89.60,scattered clouds,5.41,100.34,
25,Nelson Bay,AU,77.00,scattered clouds,-32.72,152.15,
28,Vaini,TO,75.20,broken clouds,-21.20,-175.20,
30,Kashi,CN,75.20,dust,39.45,75.98,
32,Saint-Pierre,RE,84.20,few clouds,-21.34,55.48,
33,Salamiyah,SY,76.73,few clouds,35.01,37.05,


In [4]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [5]:
# Display DataFrame.
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Dingle,PH,82.08,scattered clouds,11.00,122.67,Camp Pasica
9,Chadiza,ZM,84.70,few clouds,-14.07,32.44,Sankhani Lodge
11,Avarua,CK,71.60,overcast clouds,-21.21,-159.78,Paradise Inn
17,Inhambane,MZ,87.80,broken clouds,-23.86,35.38,MHL Turismo SA
21,Georgetown,MY,89.60,scattered clouds,5.41,100.34,Cititel Penang
25,Nelson Bay,AU,77.00,scattered clouds,-32.72,152.15,Mantra Nelson Bay
28,Vaini,TO,75.20,broken clouds,-21.20,-175.20,Keleti Beach Resort
30,Kashi,CN,75.20,dust,39.45,75.98,帕米尔青年旅舍
32,Saint-Pierre,RE,84.20,few clouds,-21.34,55.48,Lindsey Hôtel
33,Salamiyah,SY,76.73,few clouds,35.01,37.05,Gondolas Forum grandchildren and grandparents


In [6]:
# -- Saving new DataFrame as CSV file to be used for Part 3. 

# Create the output file (CSV). -- Using os.path.join to avoid "/" or "\" errors.
output_data_file = os.path.join("weather_data", "WeatherPy_vacation.csv")

# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [7]:
# -- Adding the cities to a marker layer map with a 
#    pop-up marker for each city that includes:
#    - Hotel name
#    - City
#    - Country
#    - Current weather description with the maximum temperature

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]

max_temp = hotel_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))